In [1]:
import sys
import numpy as np
import pandas as pd
import scipy
import copy
import random
import math
from scipy import stats
from scipy.stats import rankdata
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDClassifier as LogisticRegression
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn import metrics, preprocessing
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown, display
np.random.seed(1)

In [2]:
cols = ['age', 'workclass', 'fnlwgt', 'education', 'education.num', 'marital', 'occupation', 'relationship', 'race', 'gender', 'capgain', 'caploss', 'hours', 'country', 'income']
df_train = pd.read_csv('adult.data', names=cols, sep=",")
df_test = pd.read_csv('adult.test', names=cols, sep=",")

**Pre-processing** (categorical to numerical)

In [3]:
 def preprocess(df):
    df.isin(['?']).sum(axis=0)

    # replace missing values (?) to nan and then drop the columns
    df['country'] = df['country'].replace('?',np.nan)
    df['workclass'] = df['workclass'].replace('?',np.nan)
    df['occupation'] = df['occupation'].replace('?',np.nan)

    # dropping the NaN rows now
    df.dropna(how='any',inplace=True)
    df['income'] = df['income'].map({'<=50K': 0, '>50K': 1}).astype(int)
    df['age'] = df['age'].apply(lambda x : 1 if x >= 45 else 0) # 1 if old, 0 if young
    df['workclass'] = df['workclass'].map({'Never-worked': 0, 'Without-pay': 1, 'State-gov': 2, 'Local-gov': 3, 'Federal-gov': 4, 'Self-emp-inc': 5, 'Self-emp-not-inc': 6, 'Private': 7}).astype(int)
    df['education'] = df['education'].map({'Preschool': 0, '1st-4th': 1, '5th-6th': 2, '7th-8th': 3, '9th': 4, '10th': 5, '11th': 6, '12th': 7, 'HS-grad': 8, 'Some-college': 9, 'Bachelors': 10, 'Prof-school': 11, 'Assoc-acdm': 12, 'Assoc-voc': 13, 'Masters': 14, 'Doctorate': 15}).astype(int)
#     df['marital'] = df['marital'].map({'Married-civ-spouse': 1, 'Divorced': 0, 'Never-married': 0, 'Separated': 0, 'Widowed': 0, 'Married-spouse-absent': 1, 'Married-AF-spouse': 1}).astype(int)
    df['marital'] = df['marital'].map({'Married-civ-spouse': 2, 'Divorced': 1, 'Never-married': 0, 'Separated': 1, 'Widowed': 1, 'Married-spouse-absent': 2, 'Married-AF-spouse': 2}).astype(int)
    df['relationship'] = df['relationship'].map({'Wife': 1 , 'Own-child': 0 , 'Husband': 1, 'Not-in-family': 0, 'Other-relative': 0, 'Unmarried': 0}).astype(int)
    df['race'] = df['race'].map({'White': 1, 'Asian-Pac-Islander': 0, 'Amer-Indian-Eskimo': 0, 'Other': 0, 'Black': 0}).astype(int)
    df['gender'] = df['gender'].map({'Male': 1, 'Female': 0}).astype(int)
    
    # process hours
    df.loc[(df['hours'] <= 40), 'hours'] = 0
    df.loc[(df['hours'] > 40), 'hours'] = 1

    df = df.drop(columns=['fnlwgt', 'education.num', 'occupation', 'country', 'capgain', 'caploss'])
    df = df.reset_index(drop=True)
    return df

df_train = preprocess(df_train)
df_test = preprocess(df_test)

X_train = copy.deepcopy(df_train)
X_train = X_train.drop(columns=['income'])
y_train = df_train['income']

X_test = copy.deepcopy(df_test)
X_test = X_test.drop(columns=['income'])
y_test = df_test['income']

**Protected, privileged**

In [4]:
# protected: 'gender'=0
# privileged: 'gender'=1

# protected: 'age'=0
# privileged: 'age'=1

**Parametric Model**

In [5]:
# size=500
# X_train = X_train[0:size]
# y_train = y_train[0:size]

X_train_orig = copy.deepcopy(X_train)
X_test_orig = copy.deepcopy(X_test)

# Scale data: regularization penalty default: ‘l2’, ‘lbfgs’ solvers support only l2 penalties. 
# Regularization makes the predictor dependent on the scale of the features.
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

clf = LogisticRegression(loss='log', warm_start=True, max_iter=100, random_state=0,
                         average=True, shuffle=False, learning_rate='constant',
                         eta0=0.05, alpha=0.03, verbose=0)

**Compute fairness metric**

In [6]:
def computeFairness(y_pred, X_test, y_test, metric): 
    fairnessMetric = 0
    protected_idx = X_test[X_test['gender']==0].index
    numProtected = len(protected_idx)
    privileged_idx = X_test[X_test['gender']==1].index
    numPrivileged = len(privileged_idx)
        
    p_protected = 0
    for i in range(len(protected_idx)):
        p_protected += y_pred[protected_idx[i]][1]
    p_protected /= len(protected_idx)
    
    p_privileged = 0
    for i in range(len(privileged_idx)):
        p_privileged += y_pred[privileged_idx[i]][1]
    p_privileged /= len(privileged_idx)
    
    # statistical parity difference
    statistical_parity = p_protected - p_privileged
    
    # equality of opportunity, or 
    # true positive rate parity
    # P(Y=1 | Y=1, G=0)- P(Y=1 | Y=1, G=1)
    true_positive_protected = 0
    actual_positive_protected = 0
    for i in range(len(protected_idx)):
        if (y_test[protected_idx[i]] == 1):
            actual_positive_protected += 1
#             if (y_pred[protected_idx[i]][1] > y_pred[protected_idx[i]][0]):
            true_positive_protected += y_pred[protected_idx[i]][1]
    tpr_protected = true_positive_protected/actual_positive_protected
    
    true_positive_privileged = 0
    actual_positive_privileged = 0
    for i in range(len(privileged_idx)):
        if (y_test[privileged_idx[i]] == 1):
            actual_positive_privileged += 1
#             if (y_pred[privileged_idx[i]][1] > y_pred[privileged_idx[i]][0]):
            true_positive_privileged += y_pred[privileged_idx[i]][1]
    tpr_privileged = true_positive_privileged/actual_positive_privileged
    
    tpr_parity = tpr_protected - tpr_privileged
    
    # equalized odds or TPR parity + FPR parity
    # false positive rate parity
    
    # predictive parity
    p_o1_y1_s1 = 0
    p_o1_s1 = 0
    for i in range(len(protected_idx)):
#         if (y_pred[protected_idx[i]][1] > y_pred[protected_idx[i]][0]):
        p_o1_s1 += y_pred[protected_idx[i]][1]
        if (y_test[protected_idx[i]] == 1):
            p_o1_y1_s1 += y_pred[protected_idx[i]][1]
    ppv_protected = p_o1_y1_s1/p_o1_s1
    
    p_o1_y1_s0 = 0
    p_o1_s0 = 0
    for i in range(len(privileged_idx)):
#         if (y_pred[privileged_idx[i]][1] > y_pred[privileged_idx[i]][0]):
        p_o1_s0 += y_pred[privileged_idx[i]][1]
        if (y_test[privileged_idx[i]] == 1):
            p_o1_y1_s0 += y_pred[privileged_idx[i]][1]
    ppv_privileged = p_o1_y1_s0/p_o1_s0
    
    predictive_parity = ppv_protected - ppv_privileged
    
    if (metric == 0):
        fairnessMetric = statistical_parity
    elif (metric == 1):
        fairnessMetric = tpr_parity
    elif (metric == 2):
        fairnessMetric = predictive_parity
        
    return fairnessMetric

**Influence of points computed using ground truth**

In [7]:
def ground_truth_influence(X_train, y_train, X_test, X_test_orig, y_test):
    clf.fit(X_train, y_train)
    y_pred = clf.predict_proba(X_test)
    spd_0 = computeFairness(y_pred, X_test_orig, y_test, 0)

    delta_spd = []
    for i in range(len(X_train)):
        X_removed = np.delete(X_train, i, 0)
        y_removed = y_train.drop(index=i, inplace=False)
        clf.fit(X_removed, y_removed)
        y_pred = clf.predict_proba(X_test)
        delta_spd_i = computeFairness(y_pred, X_test_orig, y_test, 0) - spd_0
        delta_spd.append(delta_spd_i)
    
    return delta_spd

**Loss function** (Log loss for logistic regression)

In [8]:
def logistic_loss(y_true, y_pred):
    loss = 0
    for i in range(len(y_true)):
        if (y_pred[i][1] != 0 and y_pred[i][0] != 0):
            loss += - y_true[i] * math.log(y_pred[i][1]) - (1 - y_true[i]) * math.log(y_pred[i][0])
    loss /= len(y_true)
    return loss

**Compute Accuracy** 

In [9]:
from sklearn.metrics import accuracy_score

def computeAccuracy(y_true, y_pred):
    accuracy = 0
    for i in range(len(y_true)):
        idx = y_true[i]
        if (y_pred[i][idx] > y_pred[i][1 - idx]):
            accuracy += 1
#         accuracy += y_pred[i][idx]
    accuracy /= len(y_true)
    return accuracy

**First-order derivative of loss function at z with respect to model parameters**

In [10]:
def del_L_del_theta_i(num_params, y_true, x, y_pred):
#     del_L_del_theta = np.ones((num_params, 1)) * ((1 - y_true) * y_pred[1] - y_true * y_pred[0])
    del_L_del_theta = np.ones((num_params, 1)) * (- y_true + y_pred[1])
    for j in range(1, num_params):
            del_L_del_theta[j] *=  x[j-1]
    return del_L_del_theta

**Hessian: Second-order partial derivative of loss function with respect to model parameters**

In [11]:
# def hessian_one_point(num_params, x, y_pred):
#     H = np.ones((num_params, num_params)) * (y_pred[0] * y_pred[1])
#     for i in range(1, num_params):
#         for j in range(i + 1):
#             if j == 0:
#                 H[i][j] *= x[i-1]
#             else:
#                 H[i][j] *= x[i-1] * x[j-1] 
#     i_lower = np.tril_indices(num_params, -1)
#     H.T[i_lower] = H[i_lower]     
#     return H

**First-order derivative of $P(y \mid \textbf{x})$ with respect to model parameters**

In [12]:
def del_f_del_theta_i(num_params, x, y_pred):
    del_f_del_theta = np.ones((num_params, 1)) * (y_pred[0] * y_pred[1])
    for j in range(1, num_params):
            del_f_del_theta[j] *=  x[j-1]
    return del_f_del_theta

**Computing $v=\nabla($Statistical parity difference$)$**

In [13]:
# Return v = del(SPD)/del(theta)
def del_spd_del_theta(num_params, X_test_orig, X_test, y_pred):
    del_f_protected = np.zeros((num_params, 1))
    del_f_privileged = np.zeros((num_params, 1))
    numPrivileged = X_test_orig['gender'].sum()
    numProtected = len(X_test_orig) - numPrivileged
    for i in range(len(X_test)):
        del_f_i = del_f_del_theta_i(num_params, X_test[i], y_pred[i])
        if X_test_orig.iloc[i]['gender'] == 1: #privileged
            del_f_privileged = np.add(del_f_privileged, del_f_i)
        elif X_test_orig.iloc[i]['gender'] == 0:
            del_f_protected = np.add(del_f_protected, del_f_i)
    del_f_privileged /= numPrivileged
    del_f_protected /= numProtected
    v = np.subtract(del_f_protected, del_f_privileged)
    return v

**Computing $v=\nabla($TPR parity difference$)$**

In [14]:
# Return v = del(TPR_parity)/del(theta)
def del_tpr_parity_del_theta(num_params, X_test_orig, X_test, y_pred, y_test):
    del_f_protected = np.zeros((num_params, 1))
    del_f_privileged = np.zeros((num_params, 1))
    
    protected_idx = X_test_orig[X_test_orig['gender']==0].index
    privileged_idx = X_test_orig[X_test_orig['gender']==1].index

    actual_positive_privileged = 0
    for i in range(len(privileged_idx)):
        if (y_test[privileged_idx[i]] == 1):
            actual_positive_privileged += 1
#             if (y_pred[privileged_idx[i]][1] > y_pred[privileged_idx[i]][0]):
            del_f_i = del_f_del_theta_i(num_params, X_test[privileged_idx[i]], y_pred[privileged_idx[i]])
            del_f_privileged = np.add(del_f_privileged, del_f_i)
    del_f_privileged /= actual_positive_privileged
    
    actual_positive_protected = 0
    for i in range(len(protected_idx)):
        if (y_test[protected_idx[i]] == 1):
            actual_positive_protected += 1
#             if (y_pred[protected_idx[i]][1] > y_pred[protected_idx[i]][0]):
            del_f_i = del_f_del_theta_i(num_params, X_test[protected_idx[i]], y_pred[protected_idx[i]])
            del_f_protected = np.add(del_f_protected, del_f_i)
    del_f_protected /= actual_positive_protected

    v = np.subtract(del_f_protected, del_f_privileged)
    return v

**Computing $v=\nabla($Predictive parity difference$)$**

In [15]:
# Return v = del(Predictive_parity)/del(theta)
def del_predictive_parity_del_theta(num_params, X_test_orig, X_test, y_pred, y_test):
    del_f_protected = np.zeros((num_params, 1))
    del_f_privileged = np.zeros((num_params, 1))
    
    protected_idx = X_test_orig[X_test_orig['gender']==0].index
    privileged_idx = X_test_orig[X_test_orig['gender']==1].index

    u_dash_protected = np.zeros((num_params, 1))
    v_protected = 0
    v_dash_protected = np.zeros((num_params, 1))
    u_protected = 0
    for i in range(len(protected_idx)):
        del_f_i = del_f_del_theta_i(num_params, X_test[protected_idx[i]], y_pred[protected_idx[i]])
#         if (y_pred[protected_idx[i]][1] > y_pred[protected_idx[i]][0]):
        v_protected += y_pred[protected_idx[i]][1]
        v_dash_protected = np.add(v_dash_protected, del_f_i)
        if (y_test[protected_idx[i]] == 1):
            u_dash_protected = np.add(u_dash_protected, del_f_i)
            u_protected += y_pred[protected_idx[i]][1]
    del_f_protected = (u_dash_protected * v_protected - u_protected * v_dash_protected)/(v_protected * v_protected)
    
    u_dash_privileged = np.zeros((num_params, 1))
    v_privileged = 0
    v_dash_privileged = np.zeros((num_params, 1))
    u_privileged = 0
    for i in range(len(privileged_idx)):
        del_f_i = del_f_del_theta_i(num_params, X_test[privileged_idx[i]], y_pred[privileged_idx[i]])
#         if (y_pred[privileged_idx[i]][1] > y_pred[privileged_idx[i]][0]):
        v_privileged += y_pred[privileged_idx[i]][1]
        v_dash_privileged = np.add(v_dash_privileged, del_f_i)
        if (y_test[privileged_idx[i]] == 1):
            u_dash_privileged = np.add(u_dash_privileged, del_f_i)
            u_privileged += y_pred[privileged_idx[i]][1]
    del_f_privileged = (u_dash_privileged * v_privileged - u_privileged * v_dash_privileged)/(v_privileged * v_privileged)
    
    v = np.subtract(del_f_protected, del_f_privileged)
    return v

**Stochastic estimation of Hessian vector product (involving del fairness): $H_{\theta}^{-1}v = H_{\theta}^{-1}\nabla_{\theta}f(z, \theta) = v + [I - \nabla_{\theta}^2L(z_{s_j}, \theta^*)]H_{\theta}^{-1}v$**

In [16]:
# # Uniformly sample t points from training data 
# def hessian_vector_product(num_params, n, size, v, hessian_all_points):
#     if (size > n):
#         size = n
#     sample = random.sample(range(n), size)
#     hinv_v = copy.deepcopy(v)
#     for idx in range(size):
#         i = sample[idx]
#         hessian_i = hessian_all_points[i]
#         hinv_v = np.matmul(np.subtract(np.identity(num_params), hessian_i), hinv_v)
#         hinv_v = np.add(hinv_v, v)
#     return hinv_v

**First-order influence computation**

In [17]:
# def first_order_influence(del_L_del_theta, hinv_v, n):
#     infs = []
#     for i in range(n):
#         inf = -np.dot(del_L_del_theta[i].transpose(), hinv_v)
#         inf *= -1/n
#         infs.append(inf[0][0].tolist())
#     return infs

**Second-order influence computation for a group of points in subset U**

In [18]:
# def second_order_influence(X_train, U, size, del_L_del_theta, hessian_all_points):
#     u = len(U)
#     s = len(X_train)
#     p = u/s
#     c1 = (1 - 2*p)/(s * (1-p)**2)
#     c2 = 1/((s * (1-p))**2)
#     num_params = len(del_L_del_theta[0])
#     del_L_del_theta_hinv = np.zeros((num_params, 1))
#     del_L_del_theta_sum = np.zeros((num_params, 1))
#     hessian_U = np.zeros((num_params, num_params))
#     for i in range(u):
#         idx = U[i]
#         hessian_U = np.add(hessian_U, s * hessian_all_points[idx])
#         del_L_del_theta_sum = np.add(del_L_del_theta_sum, del_L_del_theta[idx])
    
#     hinv_del_L_del_theta= np.matmul(hinv_exact, del_L_del_theta_sum)
#     hinv_hessian_U = np.matmul(hinv_exact, hessian_U)
#     term1 = c1 * hinv_del_L_del_theta
#     term2 = c2 * np.matmul(hinv_hessian_U, hinv_del_L_del_theta)
#     sum_term = np.add(term1, term2)
#     return sum_term

**Metrics: Initial state**

In [33]:
clf.fit(X_train, y_train)

num_params = len(clf.coef_.transpose()) + 1 #weights and intercept; params: clf.coef_, clf.intercept_
y_pred_test = clf.predict_proba(X_test)
y_pred_train = clf.predict_proba(X_train)

spd_0 = computeFairness(y_pred_test, X_test_orig, y_test, 0)
print("Initial statistical parity: ", spd_0)

tpr_parity_0 = computeFairness(y_pred_test, X_test_orig, y_test, 1)
print("Initial TPR parity: ", tpr_parity_0)

predictive_parity_0 = computeFairness(y_pred_test, X_test_orig, y_test, 2)
print("Initial predictive parity: ", predictive_parity_0)

loss_0 = logistic_loss(y_test, y_pred_test)
print("Initial loss: ", loss_0)

accuracy_0 = computeAccuracy(y_test, y_pred_test)
print("Initial accuracy: ", accuracy_0)

Initial statistical parity:  -0.19373309221120977
Initial TPR parity:  -0.180473374483813
Initial predictive parity:  -0.18622824805920268
Initial loss:  0.3978614128173742
Initial accuracy:  0.801394422310757


**Pre-compute: (1) Hessian (2) del_L_del_theta for each training data point**

In [34]:
del_L_del_theta = []
for i in range(int(len(X_train))):
    del_L_del_theta.insert(i, del_L_del_theta_i(num_params, y_train[i], X_train[i], y_pred_train[i]))

# hessian_all_points = []
# for i in range(len(X_train)):
#     hessian_all_points.insert(i, hessian_one_point(num_params, X_train[i], y_pred_train[i])
#                               /len(X_train))

*Select delta fairness function depending on selected metric*

In [22]:
metric = 0
if metric == 0:
    v1 = del_spd_del_theta(num_params, X_test_orig, X_test, y_pred_test)
elif metric == 1:
    v1 = del_tpr_parity_del_theta(num_params, X_test_orig, X_test, y_pred_test, y_test)
elif metric == 2:
    v1 = del_predictive_parity_del_theta(num_params, X_test_orig, X_test, y_pred_test, y_test)

*H^{-1} computation*

In [23]:
# hexact = 1
# if hexact == 1: 
#     H_exact = np.zeros((num_params, num_params))
#     for i in range(len(X_train)):
#         H_exact = np.add(H_exact, hessian_all_points[i])
#     hinv_exact = np.linalg.pinv(H_exact) 
#     hinv_v = np.matmul(hinv_exact, v1)
# else: #using Hessian vector product
#     size_hvp = int(len(X_train) * .01)
#     hinv_v = hessian_vector_product(num_params, len(X_train), size_hvp, v1, hessian_all_points)

**First-order influence of each training data point**

In [24]:
# infs_1 = first_order_influence(del_L_del_theta, hinv_v, len(X_train))

# Repairs

In [215]:
def del_L_del_delta_i(num_params, x, y_pred, params, y_true):
    del_L_del_delta = np.ones((num_params - 1, num_params)) * (y_pred[0] * y_pred[1])
    for i in range(num_params - 1):
        for j in range(num_params):
            del_L_del_delta[i][j] *= params[i]
            if j != 0:
                del_L_del_delta[i][j] *=  x[j - 1]
                if j == i:
                    del_L_del_delta[i][j] += y_pred[1] - y_true
    return del_L_del_delta

In [216]:
def repair(idx, numIter, learningRate):
    params = np.concatenate([clf.intercept_, clf.coef_[0]], axis=0)
    clf.fit(X_train, y_train)
    y_pred_test = clf.predict_proba(X_test)
    
    X_p = copy.deepcopy(X_train[idx])
    y_p_true = copy.deepcopy(y_train[idx]) 

    del_L_del_delta = np.zeros((num_params - 1, num_params))

    random.seed(0) # seed random number generator
    delta_new = np.zeros((num_params - 1, 1))
    for i in range(len(delta_new)):
        delta_new[i] = random.random()
    threshold = 0.05
    delta_old = -1 * np.ones((num_params - 1, 1))
#     v1 = del_spd_del_theta(num_params, X_test_orig, X_test, y_pred_test)

    num_iter = 0
    del_L_del_theta_Xp = np.zeros((num_params, 1))
    for i in range(len(idx)):
        del_L_del_theta_Xp = np.add(del_L_del_theta_Xp, del_L_del_theta_i(num_params, y_train[idx[i]], X_train[idx[i]], y_pred_train[idx[i]]))
    obj_old = np.dot(del_L_del_theta_Xp.transpose(), v1)
    obj_new = obj_old + 0.1
    print(obj_old, obj_new)
    
    while (obj_new > obj_old):
#     while True:
#     while ((num_iter < 100) or (obj_new - obj_old > threshold)) :
        obj_old = obj_new
#         print(num_iter)
        num_iter += 1
        for i in range(len(idx)):
            x = np.zeros((len(X_train[idx[i]]), 1))
            for j in range(len(x)):
                x[j] = X_p[i][j]
            x = np.add(x, delta_new)
            x_ = [x[p][0] for p in range(len(x))]
            y_pred = clf.predict_proba([x_])
            del_L_del_delta_i_ = del_L_del_delta_i(num_params, x, y_pred[0], params, y_train[idx[i]])/len(idx)
            del_L_del_delta = np.add(del_L_del_delta, del_L_del_delta_i_)
        
        delta_old = delta_new
        delta_new = np.add(delta_new, (learningRate/num_iter) * np.dot(del_L_del_delta, v1))
        print(delta_new, (learningRate/num_iter) * np.dot(del_L_del_delta, v1))
        X_train_perturbed = copy.deepcopy(X_train)
        for i in range(len(idx)):
            for j in range(len(delta_new)):
                X_train_perturbed[idx[i]][j] += delta_new[j]
        del_L_del_theta_Xp = np.zeros((num_params, 1))
        for i in range(len(idx)):
            del_L_del_theta_Xp = np.add(del_L_del_theta_Xp, del_L_del_theta_i(num_params, y_train[idx[i]], X_train_perturbed[idx[i]], y_pred_train[idx[i]]))
        obj_new = np.dot(del_L_del_theta_Xp.transpose(), v1)
        print(obj_old, obj_new)
#         if (numIter>=10):
#         if ((num_iter > 10) & (obj_new - obj_old < threshold)):
#         if ((num_iter > 2) & (obj_new < obj_old)):
        if ((obj_new < obj_old)):
#             print((obj_new - obj_old))
            return delta_old
    return delta_new

In [234]:
clf.fit(X_train, y_train)
y_pred_test = clf.predict_proba(X_test).astype(np.float32)
X_p = copy.deepcopy(X_train[idx]).astype(np.float32)
y_p_true = copy.deepcopy(y_train[idx]) .astype(np.float32)

del_L_del_delta = np.zeros((num_params - 1, num_params)).astype(np.float32)

random.seed(0) # seed random number generator
delta_new = np.zeros((num_params - 1, 1))
for i in range(len(delta_new)):
    delta_new[i] = random.random()
threshold = 0.05
delta_old = -1 * np.ones((num_params - 1, 1)).astype(np.float32)

num_iter = 0
del_L_del_theta_Xp = np.zeros((num_params, 1)).astype(np.float32)
for i in range(len(idx)):
    del_L_del_theta_Xp = np.add(del_L_del_theta_Xp, del_L_del_theta_i(num_params, y_train[idx[i]], X_train[idx[i]], y_pred_train[idx[i]]))
obj_old = np.dot(del_L_del_theta_Xp.transpose(), v1).astype(np.float32)
obj_new = obj_old + 0.1

while (obj_new > obj_old):
    obj_old = obj_new.astype(np.float32)
    num_iter += 1
    for i in range(len(idx)):
        x = np.zeros((len(X_train[idx[i]]), 1))
        for j in range(len(x)):
            x[j] = X_p[i][j]
        x = np.add(x, delta_new)
        x_ = [x[p][0] for p in range(len(x))]
        y_pred = clf.predict_proba([x_]).astype(np.float32)
        del_L_del_delta_i_ = del_L_del_delta_i(num_params, x, y_pred[0], clf.coef_[0], y_train[idx[i]])/len(idx)
        del_L_del_delta = np.add(del_L_del_delta, del_L_del_delta_i_).astype(np.float32)


In [236]:
del_L_del_delta[:, -1]*len(idx)

array([ 1.6220065 , -0.41589084,  4.672706  ,  3.6291907 ,  4.406761  ,
        0.5765291 ,  1.015229  ,  2.3320346 ], dtype=float32)

In [219]:
# clf.fit(X_train, y_train)
# i = 0
# x = np.zeros((len(X_train[idx[i]]), 1))
# for j in range(len(x)):
#     x[j] = X_p[i][j]
# x = np.add(x, delta_new)
# x_ = [x[p][0] for p in range(len(x))]
# y_pred = clf.predict_proba([x_])

In [259]:
del_L_del_theta_i(num_params, y_train[idx[0]], x_, y_pred[0])

array([[ 0.38038552],
       [ 0.88456639],
       [ 0.48002878],
       [ 0.6296025 ],
       [ 0.03082164],
       [-0.15646913],
       [ 0.30764   ],
       [-0.25090479],
       [-0.13656569]])

In [256]:
coef = np.concatenate([clf.intercept_, clf.coef_[0]], axis=0).astype(np.float32)
i = 0
x = X_train[idx[i]].reshape(-1, 1).astype(np.float32)
x = np.add(x, delta_new).astype(np.float32)
x_ = [x[p][0] for p in range(len(x))]
y_pred = clf.predict_proba([x_]).astype(np.float32)

del_L_del_delta = np.ones((num_params,)).astype(np.float32) * (y_pred[0][0] * y_pred[0][1]).astype(np.float32)
for j in range(num_params):
    del_L_del_delta[j] *= coef[i].astype(np.float32)
    if j != 0:
        del_L_del_delta[j] *=  x_[j - 1].astype(np.float32)
        if j == i:
            del_L_del_delta[j] += y_pred[1].astype(np.float32) - y_true.astype(np.float32)
del_L_del_delta

array([-0.37367892, -0.8689706 , -0.47156537, -0.61850196, -0.03027822,
        0.15371041, -0.302216  ,  0.24648108,  0.1341579 ], dtype=float32)

In [223]:
[ 0.38038546, 0.88456625,  0.48002872,  0.62960243,  0.03082163, -0.15646912, 0.30763996, -0.25090474, -0.13656567]

[0.38038546,
 0.88456625,
 0.48002872,
 0.62960243,
 0.03082163,
 -0.15646912,
 0.30763996,
 -0.25090474,
 -0.13656567]

In [225]:
idx = X_train_orig[
    (X_train_orig['gender'] == 0)
    & (X_train_orig['relationship'] == 0)
    & (X_train_orig['education'] == 12)
    & (X_train_orig['race'] == 1)
    ].index 
# v_pert = repair(idx, numIter, learningRate)
v_pert = repair(idx, 100, 1)

[[9.68013652]] [[9.78013652]]
[[0.80043428]
 [0.75933796]
 [0.41798911]
 [0.27835772]
 [0.49754156]
 [0.38696803]
 [0.7796236 ]
 [0.24913542]] [[-0.04398757]
 [ 0.00138355]
 [-0.00258247]
 [ 0.01944097]
 [-0.01373316]
 [-0.01796611]
 [-0.00417498]
 [-0.0541773 ]]
[[9.78013652]] [[2.53398944]]


In [171]:
v_pert

array([[0.84442185],
       [0.7579544 ],
       [0.42057158],
       [0.25891675],
       [0.51127472],
       [0.40493414],
       [0.78379859],
       [0.30331273]])

In [56]:
res_0 = []
res_1 = []
res_2 = []
res_3 = []
res_4 = []
res_5 = []
res_6 = []
res_7 = []

X_train_copy = copy.deepcopy(X_train)
numCols = len(X_train[0])
for ix in range(len(idx)):
    X_train_pert = np.zeros((len(X_train[idx[ix]]), 1))
    for i in range(len(X_train[idx[ix]])):
        X_train_pert[i] = X_train[idx[ix]][i] + v_pert[i]

    x0 = np.random.rand(1,numCols)
    mins = []
    maxs = []
    numCols = len(X_train[0])
    for i in range(numCols):
        mins.insert(i, min(X_train[:,i]))
        maxs.insert(i, max(X_train[:,i]))

    from scipy.optimize import Bounds, minimize
    bounds = Bounds(mins, maxs)

    f = lambda x: np.linalg.norm(x - X_train_pert)

    x0 = np.random.rand(numCols)
    res = minimize(f, x0, method='trust-constr', 
    #                jac=rosen_der, hess=rosen_hess,
    #                constraints=[linear_constraint, nonlinear_constraint],
                   options={'verbose': 0}, bounds=bounds)
    
    for i in range(len(X_train_copy[idx[ix]])):
        X_train_copy[idx[ix]][i] += res.x[i]
        
    res_x_inv_transform = sc.inverse_transform(res.x, copy=None)
    res_0.insert(ix, res_x_inv_transform[0])
    res_1.insert(ix, res_x_inv_transform[1])
    res_2.insert(ix, res_x_inv_transform[2])
    res_3.insert(ix, res_x_inv_transform[3])
    res_4.insert(ix, res_x_inv_transform[4])
    res_5.insert(ix, res_x_inv_transform[5])
    res_6.insert(ix, res_x_inv_transform[6])
    res_7.insert(ix, res_x_inv_transform[7])
#     print(res_x_inv_transform[10], res_x_inv_transform[4], res_x_inv_transform[13], 
#           res_x_inv_transform[28], res_x_inv_transform[29], res_x_inv_transform[30],
#          res_x_inv_transform[17], res_x_inv_transform[18], res_x_inv_transform[19], res_x_inv_transform[20],
#          res_x_inv_transform[21], res_x_inv_transform[22], res_x_inv_transform[23], res_x_inv_transform[24],
#          res_x_inv_transform[25], res_x_inv_transform[26],
#           '\n')

clf.fit(X_train_copy, y_train)
y_pred_test = clf.predict_proba(X_test)

print(computeFairness(y_pred_test, X_test_orig, y_test, 0)/spd_0 - 1)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


0.005604568516486053


In [ ]:
clf.fit(X_train_copy, y_train, use_sklearn=True)
y_pred_test = clf.predict_proba(X_test)
computeFairness(y_pred_test, X_test_orig, y_test, 0, dataset)

In [56]:
import statistics
print(statistics.mode(res_6), statistics.mode(res_4)
      , statistics.mode(res_2)
      , statistics.mode(res_5)
     )

print(computeFairness(y_pred_test, X_test_orig, y_test, 0)/spd_0 - 1)
# print(computeFairness(y_pred_test, X_test_orig, y_test, 0))
# print(spd_0)

0.9051149870025552 0.7040802215752434 10.160973434050591 0.9999999610870457
0.008828702315179848
